### **Extract GEDI, Sentinel 1 and Sentinel 2 data for dataset associated with manuscript "A dataset on the structural diversity of European forests"**

  <span style="color:red">   Import packages</span>

In [ ]:
import ee
ee.Initialize()
ee.Initialize(project='XXXX') # need to specify a google cloud project here!
import geemap
Map = geemap.Map()
from IPython.display import JSON
import math
import os

<span style="color:red">  Functions </span>

In [2]:
# Filter using quality criteria
def gedi_qa(img):
    # Check for quality flag
    quality = img.select('quality_flag').eq(1)

    # Check if it's night
    night = img.select('solar_elevation').lte(0)

    # Check for no degradation
    degrade = img.select('degrade_flag').eq(0)

    # Check for low urban proportion
    urban = img.select('urban_proportion').lte(5)

    # Check for no water persistence
    water = img.select('landsat_water_persistence').eq(0)

    # Check for significant tree cover
    forest = img.select('landsat_treecover').gt(10)

    # Check for reasonable RH98 values
    rh98_70 = img.select(['rh98']).lte(100)

    # Check if it's leaf-on season
    leafon = img.select('leaf_off_flag').eq(0)

    # Check for detected modes
    detectmodes = img.select('num_detectedmodes').gt(0)

    # Check for surface flag
    surface = img.select('surface_flag').gt(0)

    # Check for minimum sensitivity
    sensitivitymin = img.select('sensitivity').gte(0.95)

    # Apply all masks
    return (img.mask(quality)
            .updateMask(night)
            .updateMask(degrade)
            .updateMask(water)
            .updateMask(urban)
            .updateMask(forest)
            .updateMask(slope.lt(20))
            .updateMask(rh98_70)
            .updateMask(leafon)
            .updateMask(detectmodes)
            .updateMask(surface)
            .updateMask(sensitivitymin))


# Terrain correction
# Implementation by Andreas Vollrath (ESA), inspired by Johannes Reiche (Wageningen)
def terrainCorrection(image):
    imgGeom = image.geometry()
    srtm = ee.Image('USGS/SRTMGL1_003').clip(imgGeom)  # 30m SRTM
    sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

    # Radar geometry (2.1.1)
    theta_i = image.select('angle')
    phi_i = (ee.Terrain.aspect(theta_i)
             .reduceRegion(ee.Reducer.mean(), theta_i.get('system:footprint'), 1000)
             .get('aspect'))

    # Terrain geometry (2.1.2)
    alpha_s = ee.Terrain.slope(srtm).select('slope')
    phi_s = ee.Terrain.aspect(srtm).select('aspect')

    # Model geometry (2.1.3)
    phi_r = ee.Image.constant(phi_i).subtract(phi_s)

    # Convert all to radians
    phi_rRad = phi_r.multiply(math.pi / 180)
    alpha_sRad = alpha_s.multiply(math.pi / 180)
    theta_iRad = theta_i.multiply(math.pi / 180)
    ninetyRad = ee.Image.constant(90).multiply(math.pi / 180)

    # Slope steepness in range (eq. 2)
    alpha_r = alpha_sRad.tan().multiply(phi_rRad.cos()).atan()

    # Slope steepness in azimuth (eq. 3)
    alpha_az = alpha_sRad.tan().multiply(phi_rRad.sin()).atan()

    # Local incidence angle (eq. 4)
    theta_lia = alpha_az.cos().multiply(theta_iRad.subtract(alpha_r).cos()).acos()
    theta_liaDeg = theta_lia.multiply(180 / math.pi)

    # Gamma_nought_flat (2.2)
    gamma0 = sigma0Pow.divide(theta_iRad.cos())
    gamma0dB = ee.Image.constant(10).multiply(gamma0.log10())
    ratio_1 = gamma0dB.select('VV').subtract(gamma0dB.select('VH'))

    # Volumetric Model
    nominator = (ninetyRad.subtract(theta_iRad).add(alpha_r)).tan()
    denominator = (ninetyRad.subtract(theta_iRad)).tan()
    volModel = nominator.divide(denominator).abs()

    # Apply model
    gamma0_Volume = gamma0.divide(volModel)
    gamma0_VolumeDB = ee.Image.constant(10).multiply(gamma0_Volume.log10())

    # Layover/shadow mask
    alpha_rDeg = alpha_r.multiply(180 / math.pi)
    layover = alpha_rDeg.lt(theta_i)

    shadow = theta_liaDeg.lt(85)

    # Calculate the ratio for RGB vis
    ratio = gamma0_VolumeDB.select('VV').subtract(gamma0_VolumeDB.select('VH'))

    output = (gamma0_VolumeDB
              .addBands(ratio)
              .addBands(alpha_r)
              .addBands(phi_s)
              .addBands(theta_iRad)
              .addBands(layover)
              .addBands(shadow)
              .addBands(gamma0dB)
              .addBands(ratio_1))

    return image.addBands(output.select(['VV', 'VH'], ['VV', 'VH']), None, True)

# Convert power to decibels
def powerToDb(img):
    # Multiply the logarithm base 10 of the image by 10 to convert to decibels
    return ee.Image(10).multiply(img.log10())


# Convert decibels to power
def dbToPower(img):
    # Raise 10 to the power of the image divided by 10 to convert to power
    return ee.Image(10).pow(img.divide(10))


# Convert image to decibels
def toDB(img):
    # Convert the image to decibels by taking the logarithm base 10 and multiplying by 10
    return ee.Image(img).log10().multiply(10.0)


# Convert image to natural units
def toNatural(img):
    # Convert decibels to natural units and preserve additional bands and properties
    return (
        ee.Image(10.0).pow(img.select('..').divide(10.0))  # Convert decibels to natural units
        .addBands(img.select('angle', 'DOY', 'Year'))  # Add angle, DOY, and Year bands
        .copyProperties(img, ['system:time_start'])  # Copy properties from the original image
    )


# Remove edges for VH band
def maskEdgeVH(img):
    # Create a mask to remove edges based on the VH band
    mask = (
        img.select(['VH'])
        .unitScale(-25, 5)  # Scale the VH band to the range [0, 255]
        .multiply(255)
        .toByte()
        .connectedComponents(ee.Kernel.rectangle(1, 1), 250)  # Identify connected components
    )
    # Update the mask of the image with the created mask and preserve properties
    return img.updateMask(mask.select(['VH'])).copyProperties(img, ['system:time_start'])


# Remove stripes for VH band
def stripeMaskVH(im):
    # Create a mask to remove stripes based on the VH band
    # VH backscatter of stripes is often less than -25dB
    mask = ee.Image(0).where(im.select(['VH']).lte(-25), 1).Not()
    # Update the mask of the image with the created mask and preserve properties
    return im.updateMask(mask).copyProperties(im, ['system:time_start'])


# Remove edges for VV band
def maskEdgeVV(img):
    # Create a mask to remove edges based on the VV band
    mask = (
        img.select(['VV'])
        .unitScale(-25, 5)  # Scale the VV band to the range [0, 255]
        .multiply(255)
        .toByte()
        .connectedComponents(ee.Kernel.rectangle(1, 1), 250)  # Identify connected components
    )
    # Update the mask of the image with the created mask and preserve properties
    return img.updateMask(mask.select(['VV'])).copyProperties(img, ['system:time_start'])


# Remove stripes for VV band
def stripeMaskVV(im):
    # Create a mask to remove stripes based on the VV band
    # VV backscatter of stripes is often less than -25dB
    mask = ee.Image(0).where(im.select(['VV']).lte(-25), 1).Not()
    # Update the mask of the image with the created mask and preserve properties
    return im.updateMask(mask).copyProperties(im, ['system:time_start'])


# Mask cloud and shadows for Sentinel 2
def maskCloudAndShadowsSR(image):
    # Select cloud probability band
    cloudProb = image.select('MSK_CLDPRB')

    # Get snow probability band
    snowProb = image.select('MSK_SNWPRB')

    # Select threshold for cloud probability
    cloud = cloudProb.lt(5)

    # Scene Classification Map - provides pixel classification map
    scl = image.select('SCL')

    # Create mask for scl shadows
    shadow = scl.eq(3)

    # Create mask for scl cirrus
    cirrus = scl.eq(10)

    # Create mask for snow
    snow = snowProb.lt(5)

    # Combine filters into final mask
    mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))

    # Return final image with updated mask
    return image.updateMask(mask)


# Calculate image percentiles
def imgperc(ind):
    # Mask for values > 0 for both NDVI and the spectral index
    res = (spindices.updateMask(spindices.select('ndvi').gt(0))
           .updateMask(spindices.select(ind).gt(0))
           .select(ind)
           .reduceRegion(reducer=ee.Reducer.percentile([1, 99]),
                         geometry=glb,
                         scale=1000,
                         maxPixels=1e13,
                         tileScale=4))
    # Get values from server side
    return res.getInfo()


# Add NDVI band to the image
def addNDVI(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename("ndvi")
    return img.addBands(ndvi).addBands(
        ee.Image.constant(ee.Number.parse(img.date().format("D"))).rename('DOY').float()
    )


# Remove DOY for max period
def removedoy_max(img):
    imageDOY = img.select('DOY').where(img.select('Year').eq(2020), img.select('DOY').add(365))
    imageDOY = imageDOY.select('DOY').where(img.select('Year').eq(2021), imageDOY.select('DOY').add(731))
    imageDOY = imageDOY.addBands(img.select('constant', 'angle', 'Year'))
    return imageDOY.updateMask(imageDOY.select('DOY').gte(DOY_Before)).updateMask(imageDOY.select('DOY').lte(DOY_After))


# Remove DOY for pre period
def removedoy_pre(img):
    imageDOY = img.select('DOY').where(img.select('Year').eq(2020), img.select('DOY').add(365))
    imageDOY = imageDOY.select('DOY').where(img.select('Year').eq(2021), imageDOY.select('DOY').add(731))
    imageDOY = imageDOY.addBands(img.select('constant', 'angle', 'Year'))
    return imageDOY.updateMask(imageDOY.select('DOY').gte(DOY_pre)).updateMask(imageDOY.select('DOY').lte(DOY_Before))


# Remove DOY for post period
def removedoy_post(img):
    imageDOY = img.select('DOY').where(img.select('Year').eq(2020), img.select('DOY').add(365))
    imageDOY = imageDOY.select('DOY').where(img.select('Year').eq(2021), imageDOY.select('DOY').add(731))
    imageDOY = imageDOY.addBands(img.select('constant', 'angle', 'Year'))
    return imageDOY.updateMask(imageDOY.select('DOY').gte(DOY_After)).updateMask(imageDOY.select('DOY').lte(DOY_post))


# Add date bands to the image
def addDates(img):
    return (
        img.addBands(ee.Image.constant(ee.Number.parse(img.date().format("D"))).rename('DOY').float())
        .addBands(ee.Image.constant(ee.Number.parse(img.date().format("Y"))).rename('Year').float())
    )


# Remove DOY for growing period
def removedoygrowing(img):
    imageDOY = img.select('DOY').where(img.select('Year').eq(2020), img.select('DOY').add(365))
    imageDOY = imageDOY.select('DOY').where(img.select('Year').eq(2021), imageDOY.select('DOY').add(731))
    imageDOY = imageDOY.addBands(img.select('constant', 'angle', 'Year'))
    return imageDOY.updateMask(imageDOY.select('DOY').gte(DOY_pre)).updateMask(imageDOY.select('DOY').lte(DOY_post))


<span style="color:red">  Load collections </span>

In [ ]:
# Export grid
globalgrid = ee.FeatureCollection('users/guidolavespa2511/Cell2DegSq') 

# bounding box for Global
glb = globalgrid.geometry().bounds()

# Hansen forest cover dataset
gfc = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")

# GEDI data
# 2A metrics (heights)
gedi_A = (ee.ImageCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY")
                                .filterBounds(glb))

# 2B metrics (pai, fhd)
gedi_B = (ee.ImageCollection("LARSE/GEDI/GEDI02_B_002_MONTHLY")
                                .filterBounds(glb))

# digital elevation model (dem)
dem = ee.Image('USGS/SRTMGL1_003').select('elevation')

# calculate slope from dem
slope = ee.Terrain.slope(dem)

# sentinel 1: PLEASE NOTE THAT THIS COLLECTION IS FILTERED EARLY IN THE WORFLOW USING Global AS A BOUNDING BOX!
s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(glb)

# sentinel 2: PLEASE NOTE THAT THIS COLLECTION IS FILTERED EARLY IN THE WORFLOW USING Global AS A BOUNDING BOX!
s2 = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(glb)

# alos palsar
alos = (ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')
                  .filter(ee.Filter.date('2019-01-01', '2020-12-31')))

<span style="color:red">  GEDI data pre-processing </span>

In [ ]:
# Heights: filter for quality criteria
gedi_A_clean = (gedi_A
                     #.map(gedi_sel)
                     .map(gedi_qa)
                     .select(['rh.*']).qualityMosaic('rh98'))
                     #.map(filter_rh25)
                     #.select(['rh.*','sensitivity']).qualityMosaic('rh98'))

# Aggreate Level 2B metrics
foliage = (gedi_B.qualityMosaic('fhd_normal')
                 #.updateMask(gedi_2020_B.qualityMosaic('fhd_normal'))
                 .select(['fhd_normal','pai','l2b_quality_flag','degrade_flag','cover']))

# --------- Internal variability metrics from L2A dataset
# Including Negative + Positive values
# skewness
skew = gedi_A_clean.reduce(ee.Reducer.skew()).rename('skew_negativevalues')

# kurtosis
kurt = gedi_A_clean.reduce(ee.Reducer.kurtosis()).rename('kurt_negativevalues')

# standard deviation
sd = gedi_A_clean.reduce(ee.Reducer.stdDev()).rename('sd_negativevalues')

# mean
mu = gedi_A_clean.reduce(ee.Reducer.mean()).rename('mu_negativevalues')

# cv
cv = sd.divide(mu).rename('cv_negativevalues')

# Including only Positive values
# convert to array image
arr = (gedi_A_clean
        .toArray())

# trick to solve the problem of pixels with certain bands masked
dummyImage = ee.Image(-3278).toArray()

arr1 = (arr.arrayMask(arr.gt(0)))

l = arr1.arrayLength(0)

arr2 = arr1.where(l.eq(0),dummyImage)

# mean
mu1 =  (arr2.arrayReduce(reducer = ee.Reducer.mean(),
                     axes = [0])
        .arrayFlatten([['0']])
        .rename('mean'))

# standard deviation
sd1 = (arr2.arrayReduce(reducer = ee.Reducer.stdDev(),
                     axes = [0])
        .arrayFlatten([['0']])
        .rename('sd'))

# coefficient of variation
cv1 = sd1.divide(mu1).rename('cv')

# skewness
skew1 = (arr2.arrayReduce(reducer = ee.Reducer.skew(),
                     axes = [0])
        .arrayFlatten([['0']])).rename('skew')


# kurtosis
kurt1 = (arr2.arrayReduce(reducer = ee.Reducer.kurtosis(),
                     axes = [0])
        .arrayFlatten([['0']])).rename('kurt')


#---------- Put everything together

# Heights: select relevant bands
#heights = (gedi_A_clean.select(['rh25','rh50','rh75','rh98','sensitivity']))
heights = (gedi_A_clean.select(['rh25','rh50','rh75','rh98']))


# Combine everything together
strmetrs = heights.addBands(foliage).addBands(skew).addBands(kurt).addBands(sd).addBands(mu).addBands(cv).addBands(skew1).addBands(kurt1).addBands(sd1).addBands(mu1).addBands(cv1)#.addBands(pavd_cv)

# Filter for quality criteria based on level 2B metrics as well as rh98 > 5
strmetrs = (strmetrs
                .updateMask(strmetrs.select('l2b_quality_flag').eq(1))
                .updateMask(strmetrs.select('degrade_flag').eq(0))
                .updateMask(strmetrs.select('rh50').gt(0))
                .updateMask(strmetrs.select('cover').gt(0.1))
                .select(['rh25','rh50','rh75','rh98','fhd_normal','pai','cover','skew_negativevalues','kurt_negativevalues','sd_negativevalues',
                         'mu_negativevalues','cv_negativevalues','skew','kurt','sd','mean','cv']))


<span style="color:red"> Hansen data pre-processing <br>

In [ ]:
# get tree cover for the year 2000
treecover = gfc.select(['treecover2000'])

# get tree cover loss band
loss = gfc.select(['loss'])

# only pixels with a tree cover greater than or equal to 30%
treecover30p = treecover.updateMask(treecover.gte(30))

# filter out pixels where there was loss
tcovernoloss = treecover30p.updateMask(loss.eq(0))

# self mask tree cover layer
sf_mask = treecover.updateMask(loss.eq(0)).gte(30).selfMask()

# calculated count of pixels for different forest
patch_count = sf_mask.connectedPixelCount(maxSize = 1024,eightConnected =  False)

# filter out small patches from cover dataset
tcovernoloss = tcovernoloss.updateMask(patch_count.gt(6))

<span style="color:red">  Sentinel 2 max NDVI This is needed in order to identify the dates when  NDVI reaches it  maximum values
<br> It is the baseline for identifying a window that can be defineded as growing season. Note that this only done for
the year 2020. <br> This is considered as a central year for the period of interest </span>

In [ ]:
# set start and end dates
startYear = 2020
endYear = 2021
startDate = '2020-01-01'
endDate = '2020-12-31'
start_date = ee.Date.fromYMD(startYear, 1, 1);
end_date   = ee.Date.fromYMD(endYear, 12, 31);

# filter by date
s2filt = (s2.filterDate("2020-01-01","2020-12-31")
         # filter by cloudy pixel percentage
         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70))
         .map(maskCloudAndShadowsSR)
          # add NDVI
         .map(addNDVI))

# create quality mosaic using the ndvi band
ndviCollectionMax = (s2filt
                    .qualityMosaic('ndvi')
                    .set('system:time_start',start_date.millis()))

# select only relevant bands
YearMaxComposite = ndviCollectionMax.select('ndvi','DOY')

<span style="color:red">  Sentinel 1 data pre-processing </span>

In [ ]:
# Import Sentinel-1 Collection
s1filt =  (s1.filterDate("2019-10-15","2021-03-15")
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .filterMetadata('resolution_meters', 'equals' , 10)
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .map(addDates))

# apply terrain correction
s1filt1 = s1filt.map(terrainCorrection)

# select band, mask edge and destripe (?)
s1VH = (s1filt1.select(['VH','angle','DOY','Year'])
              .map(maskEdgeVH)
              .map(stripeMaskVH)
              .map(toNatural))

# select band, mask edge and destripe (?)
s1VV = (s1filt1.select(['VV','angle','DOY','Year'])
              .map(maskEdgeVV)
              .map(stripeMaskVV)
              .map(toNatural))

##### General settings for creating Sentinel 1 composites

In [ ]:
# create shifts in the day of the YEAR
DOY_Max = YearMaxComposite.select('DOY').add(365)
DOY_Before = YearMaxComposite.select('DOY').add(365).subtract(30)
DOY_After = YearMaxComposite.select('DOY').add(365).add(30)
# substract and add 60 days. This is needed for the calculations of bimonthly composites shifted of 2 months from the window of the maximum
DOY_pre = DOY_Before.subtract(60)
DOY_post = DOY_After.add(60)


<span style="color:red">  Create Sentinel 1 backscatter growing season averages </span>

In [ ]:
# create composite for VH band
VH_mean_gs = (toDB(s1VH.map(removedoygrowing)
               .mean())
               .select('constant')
               .rename('VH_gs')
               .updateMask(tcovernoloss.gt(0)))

# create composite for VV band
VV_mean_gs = (toDB(s1VV.map(removedoygrowing)
               .mean())
               .select('constant')
               .rename('VV_gs')
               .updateMask(tcovernoloss.gt(0)))

S1comb_gs = VH_mean_gs.addBands(VV_mean_gs)

<span style="color:red">  Create Sentinel 1 backscatter growing season standard deviation </span>

In [ ]:
VH_std = (s1VH.map(removedoygrowing)
     .map(toDB)
     .reduce(ee.Reducer.stdDev())
     .select('constant_stdDev')
     .rename('VH_std')
     .updateMask(tcovernoloss.gt(0)))

VV_std = (s1VV.map(removedoygrowing)
     .map(toDB)
     .reduce(ee.Reducer.stdDev())
     .select('constant_stdDev')
     .rename('VV_std')
     .updateMask(tcovernoloss.gt(0)))

<span style="color:red">  Create Sentinel 1 backscatter bimonthly composites. The final result includes 3 composites: <br>
i) around the maximum of the growing season ii) two three months before and three months after </span>

In [ ]:
#---- VV
# composite around the maximum (+-30)
VV_around_max = (toDB(s1VV.map(removedoy_max).mean())
                    .select('constant')
                    .rename('VV_max')
                    .updateMask(tcovernoloss.gt(0)))

# composite 2 months before the maximum - 1 month
VV_pre = (toDB(s1VV.map(removedoy_pre).mean())
                      .select('constant')
                      .rename('VV_pre')
                      .updateMask(tcovernoloss.gt(0)))

# composite 2 months after the maximum + 1 month
VV_post = (toDB(s1VV.map(removedoy_post)
                    .mean())
                    .select('constant')
                    .rename('VV_post')
                    .updateMask(tcovernoloss.gt(0)))

# create a single multiband image (for VV band)
VV_bimon = VV_around_max.addBands(VV_pre).addBands(VV_post)

#---- VH
# composite around the maximum (+-30)
VH_around_max = (toDB(s1VH.map(removedoy_max)
                        .mean())
                        .select('constant')
                        .rename('VH_max')
                        .updateMask(tcovernoloss.gt(0)))

# composite 2 months before the maximum - 1 month
VH_pre = (toDB(s1VH.map(removedoy_pre)
                  .mean())
                  .select('constant')
                  .rename('VH_pre')
                  .updateMask(tcovernoloss.gt(0)))

# composite 2 months after the maximum + 1 month
VH_post = (toDB(s1VH.map(removedoy_post)
                   .mean())
                   .select('constant')
                   .rename('VH_post')
                   .updateMask(tcovernoloss.gt(0)))

# create a single multiband image (for VH band)
VH_bimon = VH_around_max.addBands(VH_pre).addBands(VH_post)

<span style="color:red">  Average of Sentinel1 backscatter values around a 3 x 3 neighbourhood </span>

In [ ]:
S1gs_mean = S1comb_gs.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

VV_bimon_mean = VV_bimon.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

VH_bimon_mean = VH_bimon.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

S1_VV_std_mean = VV_std.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

S1_VH_std_mean = VH_std.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

<span style="color:red">  Sentinel 1 texture metrics (angular second moment, entropy and dissimilarity)</span>

In [ ]:
# calculate texture metrics using min and max as inputs
glcmVHG = (VH_mean_gs
            .abs()
            .unitScale(min(VV_percabs),max(VV_percabs))
            .multiply(255)
            .toByte()
            .glcmTexture(size = 3)
            .select(['VH_gs_ent','VH_gs_diss','VH_gs_asm']))

# calculate texture metrics using min and max as inputs
glcmVVG = (VV_mean_gs
            .abs()
            .unitScale(min(VH_percabs),max(VH_percabs))
            .multiply(255)
            .toByte()
            .glcmTexture(size = 3)
            .select(['VV_gs_ent','VV_gs_diss','VV_gs_asm']))

<span style="color:red">  Sentinel 2: data pre-processing and spectral indices </span>

In [ ]:
# filter by date
s2filt = (s2.filterDate("2020-01-01","2020-12-31")
         # filter by cloudy pixel percentage
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70))
         .map(maskCloudAndShadowsSR)
          # add NDVI
         .map(lambda image: image.addBands(image.normalizedDifference(['B8', 'B4']).rename('ndvi')))
          # add NDWI
         .map(lambda image: image.addBands(image.normalizedDifference(['B8', 'B11']).rename('ndwi')))
          # add GNDVI
         .map(lambda image: image.addBands(image.normalizedDifference(['B8', 'B3']).rename('gndvi')))
          # add  modified soil-adjusted index
          .map(lambda image:  image.addBands(image.expression('(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2',
                                                       {'NIR': image
                                                        .select('B8')
                                                        .divide(10000),'RED': image.select('B4')
                                                        .divide(10000)})
                                                        .rename('msavi')))
          # // Normalized Difference Red Edge Index
          .map(lambda image: image.addBands(image.normalizedDifference(['B8', 'B5']).rename('ndre'))))

# sdNDVI
sdNDVI = (s2filt.select('ndvi').reduce(ee.Reducer.stdDev())
          .rename('ndvi_std')
          .updateMask(tcovernoloss.gt(0)))

# combine spectral indices together
spindices = (s2filt.qualityMosaic('ndvi')
            .select('ndvi','gndvi','msavi','ndre') # quality mosaic based on ndvi and selection of indices based on max ndvi
            .addBands(s2filt.qualityMosaic('ndwi').select('ndwi'))  # quality mosaic based on ndwi and selection of indices based on max ndwi
            .updateMask(tcovernoloss.gt(0))) # mask only for forested areas which have been stable

<span style="color:red">  Sentinel 2 average value for each spectral index around a 3 x 3 neighbourhood </span>

In [ ]:
sdNDVI_mean = sdNDVI.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

spindices_mean = spindices.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

<span style="color:red">  Sentinel 2 texture metrics (angular second moment, entropy and dissimilarity) </span>

##### A note on the texture metrics: the data need to be rescaled between 0 and 1. We only retained pixels where both the value of the NDVI and a given spectral index are > 0 The rationale behind this choice is that any pixel with values < 0 is likely to contain things different from vegetation.

In [ ]:
# for the NDVI it is not necessary to use the percentiles
ndvi_text = (spindices.select('ndvi')
                     .updateMask(spindices.select('ndvi').gt(0))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('ndvi_asm','ndvi_ent','ndvi_diss'))

# green chlorophyll index
gndvi_text = (spindices.updateMask(spindices.select('ndvi').gt(0))
                     .select('gndvi')
                     # mask out values that are below the 1st and above the 99th percentiles
                     .updateMask(spindices.select('gndvi').gt(gndviperc.get('gndvi_p1')))
                     .updateMask(spindices.select('gndvi').lt(gndviperc.get('gndvi_p99')))
                     # normalise between 0 and 1
                     .unitScale(gndviperc.get('gndvi_p1'),gndviperc.get('gndvi_p99'))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('gndvi_asm','gndvi_ent','gndvi_diss'))

# modified soil-adjusted index
msavi_text = (spindices.updateMask(spindices.select('ndvi').gt(0))
                     .select('msavi')
                     # mask out values that are below the 1st and above the 99th percentiles
                     .updateMask(spindices.select('msavi').gt(msaviperc.get('msavi_p1')))
                     .updateMask(spindices.select('msavi').lt(msaviperc.get('msavi_p99')))
                     # normalise between 0 and 1
                     .unitScale(msaviperc.get('msavi_p1'),msaviperc.get('msavi_p99'))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('msavi_asm','msavi_ent','msavi_diss'))

# red-edge chlorophyll index
ndre_text = (spindices.updateMask(spindices.select('ndvi').gt(0))
                     .select('ndre')
                     # mask out values that are below the 1st and above the 99th percentiles
                     .updateMask(spindices.select('ndre').gt(ndreperc.get('ndre_p1')))
                     .updateMask(spindices.select('ndre').lt(ndreperc.get('ndre_p99')))
                     # normalise between 0 and 1
                     .unitScale(ndreperc.get('ndre_p1'),ndreperc.get('ndre_p99'))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('ndre_asm','ndre_ent','ndre_diss'))

# normalised difference water index
ndwi_text = (spindices.updateMask(spindices.select('ndvi').gt(0))
                     .select('ndwi')
                     # mask out values that are below the 1st and above the 99th percentiles
                     .updateMask(spindices.select('ndwi').gt(ndwiperc.get('ndwi_p1')))
                     .updateMask(spindices.select('ndwi').lt(ndwiperc.get('ndwi_p99')))
                     # normalise between 0 and 1
                     .unitScale(ndwiperc.get('ndwi_p1'),ndwiperc.get('ndwi_p99'))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('ndwi_asm','ndwi_ent','ndwi_diss'))


<span style="color:red">  Alos palsar data

In [ ]:
DNhh = alos.select('HH').mean().pow(2).log10().multiply(10).subtract(83)#.first() #// γ₀ = 10log₁₀(DN²) - 83.0 dB
DNhv = alos.select('HV').mean().pow(2).log10().multiply(10).subtract(83)#.mean() #// γ₀ = 10log₁₀(DN²) - 83.0 dB

In [ ]:
alos_mean = (DNhh.rename('alos_HH')
    .addBands(DNhv.rename('alos_HV'))
    .updateMask(tcovernoloss.gt(0))
    .reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3)))

<span style="color:red">  Alos palsar data texture metrics

In [ ]:
#Global
hhperc = {'hh_p1':4.634242727191889, 'hh_p99':18.870941152756455}
hvperc = {'hv_p1':10.135550169048622, 'hv_p99':29.87896390839444}

alos_hh_text = (DNhh.updateMask(tcovernoloss.gt(0))
                     .abs()
                     # normalise between 0 and 1
                     .unitScale(hhperc.get('hh_p1'),hhperc.get('hh_p99'))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('HH_asm','HH_ent','HH_diss'))


alos_hv_text = (DNhv.updateMask(tcovernoloss.gt(0))
                     .abs()
                     # normalise between 0 and 1
                     .unitScale(hhperc.get('hh_p1'),hhperc.get('hh_p99'))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('HV_asm','HV_ent','HV_diss'))

<span style="color:red"> Sentinel 1 -based coherence data </span>

In [ ]:
# 6-day coherence data for the summer
summer_VV06_data = (ee.Image("users/marcogirardello/forbiores/coherence/summer_vv_COH06_100")
                            .updateMask(tcovernoloss.gt(0))).rename(['coh_6day_summer'])

# 12-day coherence data for the summer
summer_VV12_data = (ee.Image("users/marcogirardello/forbiores/coherence/summer_vv_COH12_100")
                            .updateMask(tcovernoloss.gt(0))).rename(['coh_12day_summer'])

In [ ]:
############## Warning - mask GEDI shots - we reduce the size of the data
tcovernoloss = tcovernoloss.updateMask(strmetrs.select('rh50').gt(0))
####gedi_mask = strmetrs.select('rh50').updateMask(strmetrs.select('rh50').gt(0))

strmetrs = strmetrs.multiply(10000)
S1gs_mean = S1gs_mean.multiply(10000)
S1_VV_std_mean = S1_VV_std_mean.multiply(10000)
S1_VH_std_mean = S1_VH_std_mean.multiply(10000)
VV_bimon_mean = VV_bimon_mean.multiply(10000)
VH_bimon_mean = VH_bimon_mean.multiply(10000)
glcmVHG = glcmVHG.multiply(10000)
glcmVVG = glcmVVG.multiply(10000)
spindices_mean = spindices_mean.multiply(10000)
sdNDVI_mean = sdNDVI_mean.multiply(10000)
ndvi_text = ndvi_text.multiply(10000)
gndvitext = gndvi_text.multiply(10000)
msavi_text = msavi_text.multiply(10000)
ndre_text = ndre_text.multiply(10000)
ndwi_text = ndwi_text.multiply(10000)
alos_mean = alos_mean.multiply(10000)
alos_hh_text = alos_hh_text.multiply(10000)
alos_hv_text = alos_hv_text.multiply(10000)
aspect = aspect.multiply(10000)
slope = slope.multiply(10000)
fcover = fcover.multiply(10000)
TCD = TCD.multiply(10000)

<span style="color:red">  Put everything together </span>

In [ ]:
trainingdat = (strmetrs.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()
               .addBands(S1gs_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 mean backscatter data for growing season
               .addBands(S1_VV_std_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 temporal variability backscatter data for growing season VV polarisation
               .addBands(S1_VH_std_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 temporal variability backscatter data for growing season VH polarisation
               .addBands(VV_bimon_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 bimonthly composites VV polarisation
               .addBands(VH_bimon_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 bimonthly composites VH polarisation
               .addBands(glcmVHG.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 texture metrics VH polarisation
               .addBands(glcmVVG.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 texture metrics VV polarisation
               .addBands(spindices_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S2 mean values of spectral indices
               .addBands(sdNDVI_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S2 mean values of spectral indices
               .addBands(ndvi_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # ndvi texture metrics
               .addBands(gndvi_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # gndvi texture metrics
               .addBands(msavi_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # msavi texture metrics
               .addBands(ndre_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # rcl texture metrics
               .addBands(ndwi_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # ndwi texture metrics
               .addBands(summer_VV06_data.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # coherence summer data 6 days
               .addBands(summer_VV12_data.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # coherence summer data 12 days
               .addBands(alos_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # alos palsar mean
               .addBands(alos_hh_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # alos palsar texture HH
               .addBands(alos_hv_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # alos palsar texture HV
               .addBands(elevation.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # elevation
               .addBands(slope.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # slope
               .addBands(aspect.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # aspect
               .addBands(biomass.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # biomass Santoro (AGB 2018 v3)
               .addBands(fcover.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # fcover Copernicus
               .addBands(TCD.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # Tree cover density Copernicus
               .addBands(soil.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # soil
               .addBands(temperature.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # temperature
               .addBands(precipitation.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # precipitation
               .addBands(ET.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # The Potential Evapo-Transpiration (ET0) Database v3
               .addBands(ETsd.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # The Potential Evapo-Transpiration (ET0) std Database v3
               .addBands(aridity.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # Global Aridity Index (Global-AI) Database v3
               .addBands(grid1km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # 1 km grid
               .addBands(grid5km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # 5 km grid
               .addBands(grid10km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # 10 km grid
               .addBands(grid20km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # 20 km grid
               .addBands(grid30km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # 30 km grid
               .addBands(grid40km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # 40 km grid
               .addBands(grid50km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32())) # 50 km grid

